### Execute a linha abaixo para seguir para o diretório onde estão os arquivos que serão validados
### Lembre-se de ajustar o código de acordo com a sua máquina

In [3]:
import os
# Pegar o diretório atual para usar na próxima linha
diretorio = os.getcwd()

# Mudar (ch) para o diretório aonde estão os arquivos que serão validados
os.chdir(diretorio + '\LTM')

FileNotFoundError: [WinError 2] O sistema não pode encontrar o arquivo especificado: 'C:\\Users\\Abner Cruz\\Documents\\LTM\\LTM\\LTM'

### Execute a linha abaixo para ativar as funções que serão utilizadas na validação

In [4]:
# Função para renomear os arquivos com erro
import os
import pandas as pd
from tqdm import tqdm

def renomear_arquivo(nome_arquivo):
    nome_arquivo_splited = nome_arquivo.split(sep='.')
    os.rename(nome_arquivo, nome_arquivo_splited[0]+' ERRO.'+nome_arquivo_splited[1])

### O código abaixo validará os arquivos e renomeará aqueles que estiver com ERRO

In [60]:
lista_arquivos = os.listdir()

pbar = tqdm(total=len(lista_arquivos), desc='Processando os Arquivos')

qtde_erros_sectconfig = 0
qtde_erros_flights = 0
qtde_erros_config = 0
qtde_dias_analisados = input('Informe a quantidade de dias analisados: ')
qtde_setores = input('Informe a quantidade de setores cadastrados na BDS: ')
qtde_esperada_analise = int(qtde_dias_analisados) * 3

lista_erro_sectconfig = []
lista_erro_flights = []
lista_erro_config = []

for arquivo in lista_arquivos:
    # Validar os arquivos sect_config
    if 'sect_config' in arquivo:
        # Abrir arquivo
        df = pd.read_csv(arquivo, sep=';', header=None, low_memory=False)
        
        # Coletar certas variáveis para validação 
        qtde_linhas = len(df.index) - 1 # Subtraí 1 para não contar o cabeçalho do arquivo
        arquivo_split = arquivo.split(sep='.')
    
        # Condição para validação: número de setores * 1440 (qtde de minutos em um dia)
        if (qtde_setores * 1440) != qtde_linhas:
            #renomear_arquivo(arquivo)
            qtde_erros_sectconfig += 1
            lista_erro_sectconfig.append(arquivo)
    
    # Validar os arquivos flights
    elif 'flights' in arquivo:
        
        # Abrir o arquivo como df
        df = pd.read_csv(arquivo, sep=';', header=None, low_memory=False)
        
        # coletar certas variáveis para validação
        data_arquivo = arquivo[:4] + '-' + arquivo[4:6] + '-' + arquivo[6:8]
        qtde_linhas = len(df.index)
        
        # condição para validação: ter um dia inteiro (começar com 00:00:00 e terminar com 23:59:59 do mesmo dia
        if df.iloc[1,0] != (data_arquivo + '00:00:00') and df.iloc[qtde_linhas-1,0] != (data_arquivo + '23:59:59'):
            #renomear_arquivo(arquivo)
            qtde_erros_flights += 1
            lista_erro_flights.append(arquivo)
            
    # Validar os arquivos config
    else:
        # Abrir o arquivo como df
        df = pd.read_csv(arquivo, sep=';', header=None, low_memory=False)
        
        # Condição para validação: ter 1441 linhas (qtde de minutos em um dia + a linha do cabeçalho)
        if len(df.index) != 1441:
            #renomear_arquivo(arquivo)
            qtde_erros_config += 1
            lista_erro_config.append(arquivo)
            
    pbar.update(1)
    
pbar.close()        

Processando os Arquivos:   0%|                                                                  | 0/93 [00:00<?, ?it/s]

Informe a quantidade de dias analisados: 31
Informe a quantidade de setores cadastrados na BDS: 30


Processando os Arquivos: 100%|█████████████████████████████████████████████████████████| 93/93 [00:18<00:00,  4.98it/s]


### Execute a linha abaixo para saber a quantidade de erros encontrados por tipo de arquivo

In [36]:
print('Erros encontrados:\n' + 
      'Arquivos sect_config: ' + str(qtde_erros_sectconfig) + '\n'
      'Arquivos flights: ' + str(qtde_erros_flights) +  '\n'
      'Arquivos config: ' + str(qtde_erros_config))

Erros encontrados:
Arquivos sect_config: 31
Arquivos flights: 0
Arquivos config: 11


### Execute a linha abaixo para criar um arquivo txt com um resumo, se desejar.

In [61]:
f = open('Relato de Erros.txt', 'w')
f.write(f'Quantidade de Arquivos Analisados: {len(lista_arquivos)}' + '\n\n')
f.write(f'Quantidade Esperada: {qtde_esperada_analise}' + '\n\n')
f.write('Quantidade de Arquivos com Erros encontrada:\n' + 
        'Arquivos sect_config: ' + str(qtde_erros_sectconfig) + '\n'
        'Arquivos flights: ' + str(qtde_erros_flights) +  '\n'
        'Arquivos config: ' + str(qtde_erros_config) +  '\n')
f.write('Lista de Arquivos SectConfig:' +  '\n\n')
for arquivo in lista_erro_sectconfig:
    f.write(arquivo +  '\n')
f.write('\n')
f.write('Lista de Arquivos Flights:' +  '\n\n')
for arquivo in lista_erro_flights:
    f.write(arquivo +  '\n')
f.write('\n')
f.write('Lista de Arquivos Config:' +  '\n\n')
for arquivo in lista_erro_config:
    f.write(arquivo +  '\n')
f.write('\n')
f.close()